In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
from google.colab import files
import random
from sklearn.model_selection import train_test_split
from math import ceil

Function for uploading file to Google Colab

In [2]:
files.upload()

Saving x06Simple(1).csv to x06Simple(1) (1).csv


{'x06Simple(1).csv': b'Index,Age,Temp of Water,Length of Fish\r\n1,14,25,620\r\n2,28,25,1315\r\n3,41,25,2120\r\n4,55,25,2600\r\n5,69,25,3110\r\n6,83,25,3535\r\n7,97,25,3935\r\n8,111,25,4465\r\n9,125,25,4530\r\n10,139,25,4570\r\n11,153,25,4600\r\n12,14,27,625\r\n13,28,27,1215\r\n14,41,27,2110\r\n15,55,27,2805\r\n16,69,27,3255\r\n17,83,27,4015\r\n18,97,27,4315\r\n19,111,27,4495\r\n20,125,27,4535\r\n21,139,27,4600\r\n22,153,27,4600\r\n23,14,29,590\r\n24,28,29,1305\r\n25,41,29,2140\r\n26,55,29,2890\r\n27,69,29,3920\r\n28,83,29,3920\r\n29,97,29,4515\r\n30,111,29,4520\r\n31,125,29,4525\r\n32,139,29,4565\r\n33,153,29,4566\r\n34,14,31,590\r\n35,28,31,1205\r\n36,41,31,1915\r\n37,55,31,2140\r\n38,69,31,2710\r\n39,83,31,3020\r\n40,97,31,3030\r\n41,111,31,3040\r\n42,125,31,3180\r\n43,139,31,3257\r\n44,153,31,3214\r\n'}

Random number generator, random integer in range, and random shuffle functions

In [3]:
#THE RANDOM NUMBER GENERATOR FUNCTION IS BY DEFAULT SEEDED WITH 0 IN THE FUNCTION
#DEFINITION
def r(seed=[0], m=2**32, a=1664525, c=1013904223):
  seed[0] = (a*seed[0]+c)%m
  return seed[0]

def randint(a, b):
  return int(a+(1+b-a)*r()/2**32)

def random_shuffle(data):
  len_data = data.shape[0]
  for i in range(len_data-1,0,-1):
    j = randint(0,i+1)
    data[[i,j]] = data[[j,i]]
  return data

Reading the CSV file, skipping the header information, and removing the index column.

In [ ]:
datax06_3 = np.genfromtxt('x06Simple(1).csv', delimiter=',', skip_header=1, dtype=int)
datax06_3 = np.delete(datax06_3, 0, 1)
datax06_3

Randomizing the data.

In [ ]:
datax06_3 = random_shuffle(datax06_3)
datax06_3

Separating dataset into X and Y variables

In [6]:
X06_3 = datax06_3[:,:-1]
Y06_3 = datax06_3[:,-1]

In [ ]:
X06_3

Splitting the data into training and testing sets on a 2/3 - 1/3 basis with rounding-up.

In [8]:
X_train_3 = X06_3[:ceil(2/3*datax06_3.shape[0]),:]
X_test_3 = X06_3[ceil(2/3*datax06_3.shape[0]):,:]
Y_train_3 = Y06_3[:ceil(2/3*datax06_3.shape[0])]
Y_test_3 = Y06_3[ceil(2/3*datax06_3.shape[0]):]

Calculating the mean and standard deviation of both features from the training dataset.

In [9]:
X_train_mean_3 = np.mean(X_train_3, axis=0)
X_train_std_3 = np.std(X_train_3, axis=0, ddof=1)
print(X_train_mean_3, X_train_std_3)

[86.         27.73333333] [46.55289909  2.37709375]


Standardizing the training and data using the computed mean and standard deviation of training data

In [10]:
sX_train_3 = (X_train_3-X_train_mean_3)/X_train_std_3
sX_test_3 = (X_test_3-X_train_mean_3)/X_train_std_3

Adding the bias feature

In [11]:
bias_3 = np.ones((sX_train_3.shape[0],1))
bias_sX_train_3 = np.hstack((bias_3, sX_train_3))

bias_test_3 = np.ones((sX_test_3.shape[0],1))
bias_sX_test_3 = np.hstack((bias_test_3, sX_test_3))

In [12]:
k=1
#CREATE EMPTY ARRAY TO APPEND EACH TESTING SAMPLE'S SQUARED ERROR
y_se=np.array([])
#COUNTER TO KEEP TRACK OF ITERATION NUMBER FOR REFERRENCING ACTUAL TESTING TARGET VALUE
y_pred=0
for Xtest in bias_sX_test_3:
  #INITIALIZE EMPTY WEIGHT MATRIX TO UPDATE WITH DIAGONAL WEIGHTS
  W=np.zeros(shape=(bias_sX_train_3.shape[0],bias_sX_train_3.shape[0]))
  i=0
  for Xtrain in bias_sX_train_3:
    #COMPUTE THE DISTANCE (L1) BETWEEN THE TESTING SAMPLE AND EVERY TRAINING SAMPLE
    #UPDATE THE DIAGONAL OF THE WEIGHT MATRIX
    W[i][i]=np.exp(-np.absolute((np.subtract(Xtrain, Xtest))).sum()/k)
    i+=1
  #COMPUTE LOCAL THETA FOR THIS OBSERVATION
  theta3 = np.linalg.inv(bias_sX_train_3.T @ W @bias_sX_train_3) @ bias_sX_train_3.T @ W @ Y_train_3
  #GET PREDICTED Y VALUE
  yp = Xtest @ theta3
  #COMPUTE SQUARED ERROR FOR TESTING SAMPLE
  se=np.square(np.subtract(Y_test_3[y_pred], yp))
  #APPEND SQUARED ERROR TO ARRAY
  y_se=np.append(y_se,se)
  y_pred+=1

RMSE of all testing samples

In [13]:
y_rmse = np.sqrt(y_se.mean())
y_rmse

435.32544371752994

Comparing to closed-form linear regression model.

In [14]:
closedf_3=sklearn.linear_model.LinearRegression().fit(bias_sX_train_3, Y_train_3)
print(closedf_3.coef_)
print(closedf_3.intercept_)
y_pred_3 = closedf_3.predict(bias_sX_test_3)
print(np.sqrt(np.square(np.subtract(Y_test_3, y_pred_3)).mean()))

[   0.         1266.6599659  -329.49611507]
3126.9
616.0862025884471


We can see a sizeable decrease in the RMSE of the locally-weighted regression model.